### Pt. 4 
##### Neural Network pipeline for Big G's automotive, with the following improvements:
1. Adding the 1569 partial derate
2. Because we're adjusting across multiple onehot columns, we will get errors if we reuse old code. Need something different...

In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
#read df in
#For some reason columns (7,12,15,16,18,19,20,21,22,23,25,27,28) have mixed types. Same warning as in pt. 2. to see warning, delete low_memory=False
big_g_df=pd.read_csv('../data/big_g_pipeline_ready.csv')

C:\Users\micha\AppData\Local\Temp\ipykernel_21816\2375979798.py:3: DtypeWarning: Columns (14,17,18,20,21,22,23,24,25,27,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  big_g_df=pd.read_csv('../data/big_g_pipeline_ready.csv')


In [3]:
big_g_df=big_g_df.drop(columns=['Unnamed: 0',
                      ])

In [4]:
big_g_df.head()

,EventTimeStamp,RecordID,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,...,5862_4,6773_16,6780_3,6802_31,7321_4,7323_4,7854_2,7854_3,7854_4,1569_31
0,2015-02-21 10:47:13,1,111,17,True,2,1439,38.86,-84.63,2015-02-21 11:34:25,...,False,False,False,False,False,False,False,False,False,False
1,2015-02-21 11:40:22,6,111,17,True,1,1417,33.04,-96.18,2015-02-21 11:40:59,...,False,False,False,False,False,False,False,False,False,False
2,2015-02-21 11:40:52,7,111,17,True,2,1597,36.90,-86.44,2015-02-21 11:41:29,...,False,False,False,False,False,False,False,False,False,False
3,2015-02-21 11:42:19,11,111,17,True,1,1582,40.16,-80.15,2015-02-21 11:42:55,...,False,False,False,False,False,False,False,False,False,False
4,2015-02-21 11:14:38,15,1067,2,True,127,309,36.18,-86.70,2015-02-21 11:44:52,...,False,False,False,False,False,False,False,False,False,False


In [5]:
pd.set_option('display.max_columns', None)

In [6]:
big_g_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507342 entries, 0 to 507341
Data columns (total 245 columns):
 #    Column                     Dtype  
---   ------                     -----  
 0    EventTimeStamp             object 
 1    RecordID                   int64  
 2    spn                        int64  
 3    fmi                        int64  
 4    active                     bool   
 5    activeTransitionCount      int64  
 6    EquipmentID                int64  
 7    Latitude                   float64
 8    Longitude                  float64
 9    LocationTimeStamp          object 
 10   time_of_day                object 
 11   rolling_count_5246         float64
 12   rolling_count_1569         float64
 13   BarometricPressure         object 
 14   CruiseControlActive        bool   
 15   CruiseControlSetSpeed      float64
 16   DistanceLtd                object 
 17   EngineCoolantTemperature   object 
 18   EngineLoad                 int64  
 19   EngineOilPressure    

In [7]:
# Columns (14,17,18,20,21,22,23,24,25,27,29,30) have mixed types
#updating speed, enginetimeltd and distanceltd to use
#big_g_df['BarometricPressure']=big_g_df['BarometricPressure'].str.extract('(\d+)')
#big_g_df['EngineCoolantTemperature']=big_g_df['EngineCoolantTemperature'].str.extract('(\d+)')
#big_g_df['EngineOilPressure']=big_g_df['EngineOilPressure'].str.extract('(\d+)')
#big_g_df['FuelRate']=big_g_df['FuelRate'].str.extract('(\d+)')
#big_g_df['Speed']=big_g_df['Speed'].str.extract('(\d+)')
#big_g_df['EngineTimeLtd']=big_g_df['EngineTimeLtd'].str.extract('(\d+)')
#big_g_df['DistanceLtd']=big_g_df['DistanceLtd'].str.extract('(\d+)')

In [8]:
#ask michael about this one...
#big_g_df[big_g_df[['BarometricPressure', 
#          'EngineCoolantTemperature', 
#          'EngineOilPressure', 
#          'FuelRate', 
#          'Speed', 
#          'EngineTimeLtd', 
#          'DistanceLtd']].isnull()]

In [9]:
#big_g_df[['BarometricPressure', 
#          'EngineCoolantTemperature', 
#          'EngineOilPressure', 
#          'FuelRate', 
#          'Speed', 
#          'EngineTimeLtd', 
#          'DistanceLtd']].astype(float)

In [10]:
big_g_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507342 entries, 0 to 507341
Data columns (total 245 columns):
 #    Column                     Dtype  
---   ------                     -----  
 0    EventTimeStamp             object 
 1    RecordID                   int64  
 2    spn                        int64  
 3    fmi                        int64  
 4    active                     bool   
 5    activeTransitionCount      int64  
 6    EquipmentID                int64  
 7    Latitude                   float64
 8    Longitude                  float64
 9    LocationTimeStamp          object 
 10   time_of_day                object 
 11   rolling_count_5246         float64
 12   rolling_count_1569         float64
 13   BarometricPressure         object 
 14   CruiseControlActive        bool   
 15   CruiseControlSetSpeed      float64
 16   DistanceLtd                object 
 17   EngineCoolantTemperature   object 
 18   EngineLoad                 int64  
 19   EngineOilPressure    

In [11]:
print(list(big_g_df.columns))

['EventTimeStamp', 'RecordID', 'spn', 'fmi', 'active', 'activeTransitionCount', 'EquipmentID', 'Latitude', 'Longitude', 'LocationTimeStamp', 'time_of_day', 'rolling_count_5246', 'rolling_count_1569', 'BarometricPressure', 'CruiseControlActive', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLtd', 'FuelRate', 'IgnStatus', 'IntakeManifoldTemperature', 'LampStatus', 'Speed', 'TurboBoostPressure', 'geometry', 'distance', 'spn_combined', '1761_1', '1761_10', '1761_11', '1761_17', '1761_18', '1761_19', '1761_3', '1761_4', '1761_9', '3031_18', '3031_2', '3031_3', '3031_4', '3031_9', '3216_10', '3216_11', '3216_16', '3216_2', '3216_20', '3216_21', '3216_3', '3216_4', '3216_9', '3217_2', '3218_2', '3222_5', '3226_10', '3226_11', '3226_16', '3226_2', '3226_20', '3226_21', '3226_4', '3226_9', '3227_10', '3227_21', '3228_2', '3242_0', '3242_15', '3242_16', '3242_3', '3242_4', '3246_0'

In [12]:
features = [ 'rolling_count_5246', 'rolling_count_1569', '1761_1', '1761_10', '1761_11', '1761_17', '1761_18', '1761_19', '1761_3', '1761_4', '1761_9', '3031_18', '3031_2', '3031_3', 
            '3031_4', '3031_9', '3216_10', '3216_11', '3216_16', '3216_2', '3216_20', '3216_21', '3216_3', '3216_4', '3216_9', '3217_2', 
            '3218_2', '3222_5', '3226_10', '3226_11', '3226_16', '3226_2', '3226_20', '3226_21', '3226_4', '3226_9', '3227_10', '3227_21', 
            '3228_2', '3242_0', '3242_15', '3242_16', '3242_3', '3242_4', '3246_0', '3246_15', '3246_16', '3246_2', '3246_3', '3246_4', 
            '3251_0', '3251_10', '3251_15', '3251_16', '3251_2', '3251_3', '3251_4', '3360_11', '3360_12', '3360_19', '3360_2', '3360_9', 
            '3361_12', '3361_2', '3361_3', '3361_4', '3361_5', '3362_31', '3362_7', '3363_16', '3363_3', '3363_4', '3363_5', '3363_7', 
            '3364_10', '3364_11', '3364_18', '3364_3', '3364_9', '3480_17', '3480_2', '3480_3', '3480_4', '3482_2', '3482_3', '3482_7', 
            '3490_3', '3490_4', '3490_7', '3515_10', '3521_18', '3556_18', '3556_2', '3556_5', '3610_2', '3610_3', '3610_4', '3703_31', 
            '3720_15', '3936_14', '3936_15', '3936_16', '3936_7', '4094_18', '4094_31', '4096_31', '4331_16', '4331_18', '4334_16', 
            '4334_18', '4334_2', '4334_3', '4334_4', '4339_7', '4340_3', '4340_4', '4340_5', '4342_3', '4342_4', '4342_5', '4344_3', '4344_4',
            '4344_5', '4346_5', '4360_0', '4360_10', '4360_16', '4360_3', '4360_4', '4363_0', '4363_10', '4363_16', '4363_2', '4363_3', '4363_4',
            '4364_18', '4364_31', '4375_2', '4375_4', '4376_3', '4376_4', '4376_5', '4376_7', '4765_16', '4765_2', '4765_3', '4765_4', '4766_15',
            '4766_3', '4792_14', '4794_31', '4795_31', '4796_31', '5024_10', '5031_10', '520953_4', '521032_14', '5246_16', '5298_17',
            '5298_18', '5319_31', '5392_31', '5394_3', '5394_4', '5394_5', '5394_7', '5397_31', '5491_3', '5491_4', '5491_5', '5491_7', '5569_2',
            '5742_11', '5742_12', '5742_16', '5742_3', '5742_4', '5742_9', '5743_11', '5743_12', '5743_3', '5743_4', '5743_9', '5745_18', '5745_3',
            '5745_4', '5746_4', '5835_21', '5835_3', '5835_4', '5835_9', '5848_12', '5848_13', '5848_19', '5848_4', '5848_9', '5851_18', '5851_2',
            '5853_10', '5862_0', '5862_16', '5862_2', '5862_3', '5862_4', '6773_16', '6780_3', '6802_31', '7321_4', '7323_4', '7854_2', '7854_3',
            '7854_4']

#Searching for both full and partial derate
y = big_g_df[['5246_0', '1569_31']]

# Features selection has a few added...
X = big_g_df[features]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=321)

# Scaling
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
from keras.layers import Dense

#Start with a sequential model
model = tf.keras.Sequential()

# Then add Dense layer
# See https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
model.add(
    tf.keras.layers.Dense(units = 50,
                          input_shape = (len(features),),
                          activation = 'tanh'
                          )
)

# And end with another Dense layer as the output layer.
# It needs one node per target category
# may want to use something different than softmax, per ds article...
# https://en.wikipedia.org/wiki/Softmax_function
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))
model.add(Dense(2, activation='sigmoid'))

# Finally, compile the model
# We need to use sparse categorical crossentropy since our target is encoded as integers
# We can also give one or more metrics we want to track as we train out model.
model.compile('adam', 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
from keras.models import Sequential
from keras.layers import Dense

#From the Jason Brownlee book, I think each separate selection can be tried w/ a 
#sigmoid activation, because they have different distributions???
model = Sequential()
model.add(Dense(64, input_dim=len(features), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='sigmoid'))  # Output layer for two binary labels



In [ ]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:

# Fit model on scaled data
model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=15, batch_size=32)

Epoch 1/15
11891/11891 [==============================] - 28s 2ms/step - loss: 0.0307 - accuracy: 0.0332 - val_loss: 0.0274 - val_accuracy: 0.0158
Epoch 2/15
11891/11891 [==============================] - 28s 2ms/step - loss: 0.0275 - accuracy: 0.0118 - val_loss: 0.0273 - val_accuracy: 0.0107
Epoch 3/15
11891/11891 [==============================] - 27s 2ms/step - loss: 0.0274 - accuracy: 0.0112 - val_loss: 0.0273 - val_accuracy: 0.0115
Epoch 4/15
11891/11891 [==============================] - 28s 2ms/step - loss: 0.0275 - accuracy: 0.0106 - val_loss: 0.0273 - val_accuracy: 0.0103
Epoch 5/15
11891/11891 [==============================] - 36s 3ms/step - loss: 0.0274 - accuracy: 0.0100 - val_loss: 0.0273 - val_accuracy: 0.0103
Epoch 6/15
11891/11891 [==============================] - 33s 3ms/step - loss: 0.0274 - accuracy: 0.0097 - val_loss: 0.0273 - val_accuracy: 0.0098
Epoch 7/15
11891/11891 [==============================] - 29s 2ms/step - loss: 0.0274 - accuracy: 0.0097 - val_loss: 0

In [16]:
model.predict(X_test)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [17]:
y_pred = np.argmax(model.predict(X_test), axis = 1)
y_pred

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [18]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

NameError: name 'y_pred' is not defined